In [1]:
import os

In [ ]:

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [4]:

import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import cityscapes_labels
import utils

from models import *
from keras import losses, metrics, optimizers
from metrics import precision, dice_coef


In [5]:
!nvidia-smi

Wed Feb 28 15:23:55 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   41C    P8     7W / 120W |    354MiB /  6072MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 105...  Off  | 00000000:02:00.0 Off |                  N/A |
| 51%   26C    P8   ERR! /  75W |      2MiB /  4038MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [7]:
#target_height, target_width = 360, 480
#target_height, target_width = 360, 648
target_size = 384, 640

import config


labels = cityscapes_labels.labels
n_classes = len(labels)
batch_size = 2
epochs = 10

dataset_path = config.data_path('gta')
images_path = os.path.join(dataset_path, 'images/')
labels_path = os.path.join(dataset_path, 'labels/')

# segnet
# model = MobileUNet(target_size, n_classes)
model = SegNet(target_size, n_classes)

#model.k.load_weights('weights/SegNet_2018_02_02_11:44_(200)_finished.h5')
# model.k.load_weights('weights/MobileUNet_2018_02_13_23:26_cat_acc-0.87.hdf5')
model.k.load_weights('checkpoint/SegNet_d_save_epoch.h5')

model.summary()

model.k.compile(
    loss=losses.categorical_crossentropy,
    optimizer=optimizers.Adam(lr=0.001),
    metrics=[
        dice_coef,
        precision,
        metrics.categorical_accuracy
    ]
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_2 (Layer)              (None, 384, 640, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 384, 640, 64)      1792      
_________________________________________________________________
batch_normalization_9 (Batch (None, 384, 640, 64)      256       
_________________________________________________________________
activation_6 (Activation)    (None, 384, 640, 64)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 192, 320, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 192, 320, 128)     73856     
_________________________________________________________________
batch_normalization_10 (Batc (None, 192, 320, 128)     512       
__________

In [ ]:
!ls weights

In [15]:
from generator import data_generator
import cityscapes_labels

# testing set!
#predict_filenames = [
#    images_path + '00001.png',
#    images_path + '22001.png',
#    './results/cityscapes_2_img.png',
#    './results/exp1_camvid.png',
#    './results/img_aachen_000007_000019_leftImg8bit.png.png'
#]
#

predict_filenames = [
     images_path + '00003.png',
     images_path + '00005.png',
     images_path + '00008.png',
]

f, arr= plt.subplots(len(predict_filenames),2)
f.set_size_inches(20,20)

for i, filename in enumerate(predict_filenames):
    print(i, filename)

    image = cv2.imread(filename)

    print(image.shape)

    norm = data_generator.GTAGenerator.default_normalize(image, target_size)

    prediction = model.k.predict(np.array([norm]), batch_size, verbose=1)

    class_scores = prediction.reshape((target_size[0], target_size[1], n_classes))
    class_image = np.argmax(class_scores, axis=2)
    colored_class_image = utils.class_image_to_image(class_image, cityscapes_labels.trainId2label)  # there should be trainid2label

    predicting_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    arr[i][0].imshow(predicting_image)
    arr[i][1].imshow(colored_class_image)

    # im_to_save = cv2.cvtColor(colored_class_image, cv2.COLOR_RGB2BGR)

    # cv2.imwrite('results/img_'+str(filename.split('/')[-1])+'.png', image)
    # cv2.imwrite('results/pred_'+ str(filename.split('/')[-1])+'.png', im_to_save)

(0, '/home/mlyko/data/gta/images/00003.png')
(1052, 1914, 3)
1/1 [==============================] - 0s 86ms/step


(1, '/home/mlyko/data/gta/images/00005.png')
(1052, 1914, 3)
1/1 [==============================] - 0s 86ms/step


(2, '/home/mlyko/data/gta/images/00008.png')
(1052, 1914, 3)
1/1 [==============================] - 0s 87ms/step


In [ ]:
# test visualize
import keras.backend as K

predict_filenames = [
     images_path + '00003.png',
#     images_path + '00005.png',
#     images_path + '00008.png',
]

#f, arr= plt.subplots(len(predict_filenames),2)
#f.set_size_inches(20,20)


def layer_to_visualize(img_to_visualize, layer, model):
    inputs = [K.learning_phase()] + model.inputs

    _convout1_f = K.function(inputs, [layer.output])
    def convout1_f(X):
        # The [0] is to disable the training phase flag
        return _convout1_f([0] + [X])

    convolutions = convout1_f(img_to_visualize)
    convolutions = np.squeeze(convolutions)
    convolutions=np.moveaxis(convolutions, 2, 0)
    print ('Shape of conv:', convolutions.shape)
    
    filters = convolutions.shape[0]
    print("convs", filters)
    n = int(np.ceil(np.sqrt(filters)))
#    print("sqrted", n)
    # Visualization of each filter of the layer
    fig = plt.figure(figsize=(32,32))
    for i in range(filters):
        ax = fig.add_subplot(n,n,i+1)
        ax.imshow(convolutions[i], cmap='rgb')


In [ ]:
for i, filename in enumerate(predict_filenames):
    print(i, filename)

    image = cv2.imread(filename)

    print(image.shape)

    norm = data_generator.SimpleSegmentationGenerator.normalize(image, (target_height,target_width))

#    to_vis = np.array([norm])
    to_vis = np.expand_dims(norm, axis=0)
    print(to_vis.shape)
    
    layer_to_visualize(to_vis, model.k.get_layer('conv_pw_5_relu'), model.k)    
    
    layer_to_visualize(to_vis, model.k.get_layer('conv_pw_13_relu'), model.k)    

    layer_to_visualize(to_vis, model.k.get_layer('conv2d_transpose_1'), model.k)    
    layer_to_visualize(to_vis, model.k.get_layer('conv2d_transpose_4'), model.k)    

In [ ]:
!pwd